In [14]:
import numpy as np
from PEPit import PEP, Point, Expression
from PEPit.functions import (
    SmoothStronglyConvexFunction,
    SmoothStronglyConvexQuadraticFunction,
    ConvexFunction,
    ConvexLipschitzFunction,
)
from PEPit.primitive_steps import proximal_step
from PEPit.tools.expressions_to_matrices import expression_to_matrices

problem = PEP()

Certain `Point` and `Expression` objects are called as _leaf_ nodes, if it serves as a basis to expressing other `Point` or `Expressoin` object as linear combitation of those.

In [15]:
p1 = Point()
p2 = Point()
p = p1 + 2*p2

# Check if the given Point objects are leaf nodes
print(p1._is_leaf, p2._is_leaf, p._is_leaf)

# Check if the leaf nodes have a decomposition_dict which is
# only composed of single key with associated value (multiplier) being 1.
print(p1.decomposition_dict, p2.decomposition_dict)

# Check if decomposition_dict of p is of two keys with 
# associated value being 1 and 2 for p1 and p2, respectively.
print(p.decomposition_dict)

True True False
{<PEPit.point.Point object at 0x1392cb9e0>: 1} {<PEPit.point.Point object at 0x13955b4a0>: 1}
{<PEPit.point.Point object at 0x1392cb9e0>: 1, <PEPit.point.Point object at 0x13955b4a0>: 2}


Given a function `f`, the gradient evaluation and function evaluation adds leaf nodes to leaf lists.

In [20]:
# Definition of PEP() object resets the list and counters of Point and Expression
problem = PEP()
f = ConvexFunction()

# This adds xs and gs to the list of points and fs to the list of expressions
xs = f.stationary_point()
gs = f.gradient(xs)
fs = f(xs)
print(xs._is_leaf, gs._is_leaf, fs._is_leaf)

print('List of leaf Points:', Point.list_of_leaf_points)
print('List of leaf Expressions:', Expression.list_of_leaf_expressions)


True True True
List of leaf Points: [<PEPit.point.Point object at 0x1395564b0>, <PEPit.point.Point object at 0x139556720>]
List of leaf Expressions: [<PEPit.expression.Expression object at 0x139557770>]


In [36]:
problem = PEP()
f = ConvexFunction()

# Initial point x0 defined as a Point object.
print('\nLeaf Point list before defining x0:\n', Point.list_of_leaf_points)
x0 = problem.set_initial_point()
print('Only defined initial point:\n', Point.list_of_leaf_points)

# g0 not included in the leaf Point list until gradient is evaluated.
g0 = f.gradient(x0)
print('Also evaluated gradient at x0:\n', Point.list_of_leaf_points)

# Evaluating gradient automatically evaluates function value,
# therefore the f(x0) expression is added to the leaf Expression list
# even before evaluating f(x0).
print('\nLeaf Expression list before evaluating f(x0):\n', Expression.list_of_leaf_expressions)
f0 = f(x0)
print('Leaf Expression list after evaluating f(x0):\n', Expression.list_of_leaf_expressions)

problem = PEP()
f = ConvexFunction()

# This also happens vice versa, i.e., g1 is automatically added to
# the leaf Point list when f(x1) is evaluated.
print('\nLeaf Point list before defining x1:\n', Point.list_of_leaf_points)
x1 = Point()
print('Leaf Point list before evaluating f(x1):\n', Point.list_of_leaf_points)
f1 = f(x1)
print('Leaf Expression list after evaluating f(x1)\n', Expression.list_of_leaf_expressions)
print('Leaf Point list after evaluating f(x1)\n', Point.list_of_leaf_points)


Leaf Point list before defining x0:
 []
Only defined initial point:
Also evaluated gradient at x0:
 [<PEPit.point.Point object at 0x12a34ad80>, <PEPit.point.Point object at 0x139556600>]

Leaf Expression list before evaluating f(x0):
Leaf Expression list after evaluating f(x0):

Leaf Point list before defining x1:
 []
Leaf Point list before evaluating f(x1):
Leaf Expression list after evaluating f(x1)
Leaf Point list after evaluating f(x1)
 [<PEPit.point.Point object at 0x139554f50>, <PEPit.point.Point object at 0x139556a50>]


Defining the stationary point adds leaf nodes in the following sense.

* First, define `Point` object `xs` with associated gradient `gs` and `fs`, as a triplet `(xs, gs, fs)` for function `F`. Then `xs` and `fs` will be added to the leaf nodes list, whereas `gs` will be a non-leaf node with null `decomposition_dict`.

* Second, reference the `F.decomposition_dict`, which starts with `f` defined below.

* Third, iterate over the basis functions in `F.decomposition_dict` to evaluate the gradient and function value at `xs` for each basis function. Each gradient and function value will be leaf `Point` and leaf `Expression` object.

* Lastly, when reached the last basis function, add the triple `(xs, gs-(weighted_sum_of_all_previous_gradients), fs-(weighted_sum_of_all_previous_func_values))` to list of points to the last basis function.

In [68]:
problem = PEP()

f = SmoothStronglyConvexFunction(mu=0.0, L=1.0)
g = ConvexLipschitzFunction(M=1,reuse_gradient=False)
F = f + 0.5*g
print('F is defined as a linear combination of f and g:\n', F.decomposition_dict)

xs = F.stationary_point()
print('\nThe leaf Point list after defining xs:\n', Point.list_of_leaf_points)
print('The leaf Expression list after defining xs:\n', Expression.list_of_leaf_expressions)

gs_f = f.gradient(xs)
print('\nThe leaf Point list after evaluating f(xs):\n', Point.list_of_leaf_points)
print('The leaf Expression list after evaluating f(xs):\n', Expression.list_of_leaf_expressions)
print(gs_f._is_leaf, gs_f.decomposition_dict)

# When reuse_gradient is True, then below is not a leaf node,
# whereas reuse_gradient being False makes gs_g a leaf node.
gs_g = g.gradient(xs)
fs_g = g(xs)
print('\nThe leaf Point list after evaluating g(xs):\n', Point.list_of_leaf_points)
print('The leaf Expression list after evaluating g(xs):\n', Expression.list_of_leaf_expressions)
print(gs_g._is_leaf, fs_g._is_leaf)


F is defined as a linear combination of f and g:
 {<PEPit.functions.smooth_strongly_convex_function.SmoothStronglyConvexFunction object at 0x13955bf80>: 1, <PEPit.functions.convex_lipschitz_function.ConvexLipschitzFunction object at 0x103c02210>: 0.5}

The leaf Point list after defining xs:
 [<PEPit.point.Point object at 0x139309a90>, <PEPit.point.Point object at 0x10b597bf0>]
The leaf Expression list after defining xs:
 [<PEPit.expression.Expression object at 0x139558650>, <PEPit.expression.Expression object at 0x13955b9b0>]

The leaf Point list after evaluating f(xs):
 [<PEPit.point.Point object at 0x139309a90>, <PEPit.point.Point object at 0x10b597bf0>]
The leaf Expression list after evaluating f(xs):
 [<PEPit.expression.Expression object at 0x139558650>, <PEPit.expression.Expression object at 0x13955b9b0>]
True {<PEPit.point.Point object at 0x10b597bf0>: 1}

The leaf Point list after evaluating g(xs):
 [<PEPit.point.Point object at 0x139309a90>, <PEPit.point.Point object at 0x10b59

Then the following PEPit implementation would give the leaf `Point` and `Expression` lists as follows.

* The list of leaf `Point` objects will be:
```
```
    [ xs, f1.gradient(xs), x0, g0=f1.gradient(x0), f2.gradient(x0),
      x[1], g[1]=f2.gradient(x[1]), f1.gradient(x[1]),
      x[2], g[2]=f2.gradient(x[2]), ..., f1.gradient(x[K-1]),
      x[K], g[K]=f2.gradient(x[K]), f1.gradient(x[K]) ]
```
```

* The list of leaf `Expression` objects will be:
```
```
    [ tau, fs, f1(xs), f0=f1(x0), f2(x0),
      f[1]=f2(x[1]), f1(x[1]),
      f[2]=f2(x[2]), ..., f1(x[K-1]),
      f[K], f1(x[K]), _ ]
```
```

Then the LMIs will be represented with the Gram matrix and F-vector of the orders of each entry being as above.

In [76]:
problem = PEP()
mu = 0.1
L = 1
gamma = 0.5 / L
K = 2
lambd = 0.1

# Declare a strongly convex smooth function and a closed convex proper function
f1 = problem.declare_function(SmoothStronglyConvexQuadraticFunction, mu=mu, L=L, reuse_gradient=True)
f2 = problem.declare_function(ConvexLipschitzFunction, M=lambd, reuse_gradient=True)
func = f1 + f2

# Start by defining its unique optimal point xs = x_*
xs = func.stationary_point()        # add [xs, f1.gradient(xs)] and [func(xs), f1(xs)]

# Then define the starting point x0 of the algorithm
x0 = problem.set_initial_point()    # add [x0]
g0 = f1.gradient(x0)                # add [g0=f1.gradient(x0)] and [f1(x0)]
f0 = func(x0)                       # add [f2.gradient(x0)] and [f2(x0)]

# Set the initial constraint that is the distance between x0 and x^*
problem.set_initial_condition((x0 - xs) ** 2 <= 4)

# Run the proximal gradient method starting from x0

x = [x0 for _ in range(K+1)]
g = [g0 for _ in range(K+1)]
f = [f0 for _ in range(K+1)]

# x = x0
for k in range(K):
    # y = x - gamma * f1.gradient(x)
    # x, gx, fx = proximal_step(y, f2, gamma)

    y = x[k] - gamma * f1.gradient(x[k])                    # add [f1.gradient(x[k])] and [f1(x[k])] if k>0
    x[k+1], g[k+1], f[k+1] = proximal_step(y, f2, gamma)    # add [x[k+1], g[k+1]] and [f[k+1]]

# Set the performance metric to the distance between x and xs
# problem.set_performance_metric((x - xs) ** 2)
# problem.set_performance_metric(func(x0) - func(xs))
# problem.set_performance_metric(f[-1] - func(xs))

problem.set_performance_metric(func(x[-1]) - func(xs))      # add [f1.gradient(x[K])] and [f1(x[K])]

# Solve the PEP
# pepit_tau = problem.solve(wrapper=wrapper, solver=solver, verbose=pepit_verbose)

# problem.set_initial_condition(x[1] ** 2 <= 9)
# problem.set_initial_condition(func(x[1]) <= 9)
# problem.set_initial_condition(f2.gradient(xs) ** 2 <= 5)

pepit_tau = problem.solve(wrapper='cvxpy', solver='CLARABEL')

print(pepit_tau)

A_obj, b_obj, _ = expression_to_matrices(problem._list_of_constraints_sent_to_wrapper[0].expression)
print(A_obj, b_obj)
print(A_obj.shape, b_obj.shape)

print(len(problem._list_of_constraints_sent_to_wrapper))
for constr in problem._list_of_constraints_sent_to_wrapper[1:]:
    A_cons, b_cons, c_cons = expression_to_matrices(constr.expression)

    print(A_cons, b_cons, c_cons)

(PEPit) Setting up the problem: size of the Gram matrix: 10x10
(PEPit) Setting up the problem: performance measure is the minimum of 1 element(s)
(PEPit) Setting up the problem: Adding initial conditions and general constraints ...
(PEPit) Setting up the problem: initial conditions and general constraints (1 constraint(s) added)
(PEPit) Setting up the problem: interpolation conditions for 2 function(s)
			Function 1 : Adding 15 scalar constraint(s) ...
			Function 1 : 15 scalar constraint(s) added
			Function 1 : Adding 1 lmi constraint(s) ...
		 Size of PSD matrix 1: 5x5
			Function 1 : 1 lmi constraint(s) added
			Function 2 : Adding 16 scalar constraint(s) ...
			Function 2 : 16 scalar constraint(s) added
(PEPit) Setting up the problem: additional constraints for 0 function(s)
(PEPit) Compiling SDP
(PEPit) Calling SDP solver
(PEPit) Solver status: optimal (wrapper:cvxpy, solver: CLARABEL); optimal value: 0.4944682870823585
(PEPit) Postprocessing: solver's output is not entirely feas